In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# One-Hot

In [3]:
from sklearn import preprocessing
import xgboost as xgb

In [4]:
from math import radians, cos, sin, asin, sqrt  
  
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）  
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """  
    # 将十进制度数转化为弧度  
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine公式  
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))   
    r = 6371 # 地球平均半径，单位为公里  
    return c * r * 100

In [5]:
shop = pd.read_csv('../input/训练数据-ccf_first_round_shop_info.csv')

In [45]:
wifi_infos = pd.read_csv('./output/wifi_info.csv')
wifi_infos = wifi_infos.drop(['Unnamed: 0'], axis=1)

wifi_infos.shape
wifi_infos.head()

(11075599, 5)

,bssid,connect,index,mall_id,strength
0,b_6396480,False,0,m_1409,-67
1,b_41124514,False,0,m_1409,-86
2,b_28723327,False,0,m_1409,-90
3,b_6396479,False,0,m_1409,-55
4,b_8764723,False,0,m_1409,-90


In [46]:
wifi_infos = wifi_infos.bssid.value_counts()

In [47]:
many_wifi = []
for index in wifi_infos.index:
    if wifi_infos[index] >= 100:
        many_wifi.append(index)

In [48]:
len(many_wifi)

26559

In [12]:
train = pd.read_csv('./output/train_neg9.csv')
test = pd.read_csv('./output/test_neg10.csv')

In [40]:
train = train[['user_id', 'shop_id', 'time_stamp', 'longitude_x', 'latitude_x', 'wifi_infos', 'full_order', 'flag']]
               
train = train.merge(shop, left_on='shop_id', right_on='shop_id', how='left')

In [13]:
test = test.merge(shop, left_on='shop_id', right_on='shop_id', how='left')

test['full_order'] = test.index
test.full_order = test.full_order / 9
test.full_order = test.full_order.astype(int)

In [9]:
train.columns = ['user_id', 'shop_id', 'time_stamp', 'longitude_x', 'latitude_x', 'wifi_infos','full_order', 'flag', 'category_id', 'longitude_y', 'latitude_y', 'price', 'mall_id']
test.columns = ['user_id', 'mall_id', 'time_stamp', 'longitude_x', 'latitude_x', 'wifi_infos', 'shop_id', 'category_id', 'longitude_y', 'latitude_y', 'price', 'mall_id', 'full_order']

In [16]:
train = train[['user_id', 'shop_id', 'time_stamp', 'longitude_x', 'latitude_x', 'wifi_infos', 'category_id', 'longitude_y', 'latitude_y', 'mall_id', 'full_order', 'flag']]
test = test[['user_id', 'shop_id', 'time_stamp', 'longitude_x', 'latitude_x', 'wifi_infos', 'category_id', 'longitude_y', 'latitude_y', 'mall_id_x', 'full_order', 'flag']]
test.columns = ['user_id', 'shop_id', 'time_stamp', 'longitude_x', 'latitude_x', 'wifi_infos', 'category_id', 'longitude_y', 'latitude_y', 'mall_id', 'full_order', 'flag']

In [14]:
test.full_order = test.full_order + 1138015
test['flag'] = -1

In [17]:
train.shape
train.head()
test.shape
test.head()

(10242135, 12)

,user_id,shop_id,time_stamp,longitude_x,latitude_x,wifi_infos,category_id,longitude_y,latitude_y,mall_id,full_order,flag
0,u_23100,s_410892,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_29,113.819891,34.948559,m_2270,123,1
1,u_23100,s_428507,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_132,113.819719,34.948597,m_2270,123,0
2,u_23100,s_3699611,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_27,113.819822,34.949240,m_2270,123,0
3,u_23100,s_8146,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_49,113.819720,34.948545,m_2270,123,0
4,u_23100,s_428193,2017-08-01 15:10,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_28,113.819570,34.949528,m_2270,123,0


(4355379, 12)

,user_id,shop_id,time_stamp,longitude_x,latitude_x,wifi_infos,category_id,longitude_y,latitude_y,mall_id,full_order,flag
0,u_30097142,s_3683488,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,c_25,122.140708,39.819073,m_3916,1138015,-1
1,u_30097142,s_147921,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,c_30,122.141208,39.818696,m_3916,1138015,-1
2,u_30097142,s_312218,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,c_28,122.140651,39.819060,m_3916,1138015,-1
3,u_30097142,s_10425,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,c_29,122.140955,39.819088,m_3916,1138015,-1
4,u_30097142,s_3997599,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...,c_28,122.140818,39.819379,m_3916,1138015,-1


In [22]:
# full_data = pd.concat([train, test])

full_data.shape
full_data.head()

(14597514, 14)

,user_id,shop_id,longitude_x,latitude_x,wifi_infos,category_id,longitude_y,latitude_y,mall_id,full_order,flag,distance,time_hour,time_week
0,u_23100,s_410892,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_29,113.819891,34.948559,m_2270,123,1,1.716082,15,1
1,u_23100,s_428507,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_132,113.819719,34.948597,m_2270,123,0,0.637523,15,1
2,u_23100,s_3699611,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_27,113.819822,34.949240,m_2270,123,0,7.846579,15,1
3,u_23100,s_8146,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_49,113.819720,34.948545,m_2270,123,0,0.152461,15,1
4,u_23100,s_428193,113.819704,34.948541,b_3927949|-72|false;b_1639608|-37|false;b_1285...,c_28,113.819570,34.949528,m_2270,123,0,11.042684,15,1


In [19]:
%%time
full_data['distance'] = full_data.apply(lambda x : haversine(x.longitude_x, x.latitude_x, x.longitude_y, x.latitude_y), axis=1)

CPU times: user 13min, sys: 12.6 s, total: 13min 13s
Wall time: 13min 13s


In [23]:
%%time
def scaleDis(x):
        maxT = max(full_data.distance[x.name - x.name%9 : x.name - x.name%9 + 9])
        minT = min(full_data.distance[x.name - x.name%9 : x.name - x.name%9 + 9])
        return (x.distance - minT)/(maxT - minT)
    
distance = full_data.apply(scaleDis, axis=1)

/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
/usr/local/miniconda2/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.


CPU times: user 53min 40s, sys: 16.4 s, total: 53min 56s
Wall time: 56min 15s


In [24]:
distance[:20]

0     0.058967
1     0.018293
2     0.290158
3     0.000000
4     0.410689
5     0.084164
6     0.283888
7     1.000000
8     0.104307
9     0.041813
10    0.000000
11    0.680746
12    0.023712
13    0.697858
14    0.238668
15    0.281177
16    0.284073
17    1.000000
18    0.023590
19    0.000000
dtype: float64

In [25]:
full_data['dis'] = distance

In [20]:
full_data.time_stamp = pd.to_datetime(full_data.time_stamp, format="%Y-%m-%d")
full_data['time_hour'] = full_data.time_stamp.apply(lambda x : x.hour).astype(int)
full_data['time_week'] = full_data.time_stamp.apply(lambda x : x.dayofweek).astype(int)
full_data = full_data.drop(['time_stamp'], axis=1)

In [26]:
full_data.to_csv('./output/fulldata_neg9_dis.csv', index=False)

In [27]:
full_data = pd.read_csv('./output/fulldata_neg_dis.csv')
mall_ids = set(full_data.mall_id)

full_data.shape
full_data.head()

(8109730, 8)

,user_id,mall_id,wifi_infos,shop_id,flag,time_hour,time_week,distance
0,u_376,m_1409,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718,1,21,6,4.902234
1,u_5847255,m_1409,b_8764725|-72|false;b_5857370|-64|false;b_5074...,s_2871718,0,13,6,3.429618
2,u_35745402,m_1409,b_56326320|-46|false;b_30230709|-46|false;b_56...,s_2871718,0,17,3,1.197266
3,u_29400269,m_1409,b_28723327|-70|false;b_20517758|-39|false;b_28...,s_2871718,0,19,2,7.258564
4,u_11893024,m_1409,b_5577440|-46|true;b_5577439|-52|false;b_22985...,s_2871718,0,19,4,7.216426


In [10]:
train = full_data[full_data.flag != -1]
test = full_data[full_data.flag == -1]

del full_data

train.shape
train.head()
test.shape
test.head()

(5690075, 9)

,user_id,mall_id,longitude,latitude,shop_id,flag,time_hour,time_week,full_order
0,376,m_1409,122.308291,32.088040,s_2871718,1,21,6,0
1,5847255,m_1409,122.308339,32.088094,s_2871718,0,13,6,1
2,35745402,m_1409,122.308446,32.088192,s_2871718,0,17,3,2
3,29400269,m_1409,122.308613,32.088048,s_2871718,0,19,2,3
4,11893024,m_1409,122.307937,32.087890,s_2871718,0,19,4,4


(4355379, 9)

,user_id,mall_id,longitude,latitude,shop_id,flag,time_hour,time_week,full_order
5690075,30097142,m_3916,122.141011,39.818847,s_3683488,-1,13,1,0
5690076,30097142,m_3916,122.141011,39.818847,s_147921,-1,13,1,1
5690077,30097142,m_3916,122.141011,39.818847,s_312218,-1,13,1,2
5690078,30097142,m_3916,122.141011,39.818847,s_10425,-1,13,1,3
5690079,30097142,m_3916,122.141011,39.818847,s_3997599,-1,13,1,4


In [28]:
def mall_wifi_dateframe(mall_id, path):
    cols = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'_cols.p','rb'))
    coo = pickle.load(open('../cjy/input/'+path+'/'+mall_id+'.p','rb'))
    coo = coo.tocoo(copy=False)
    coo_group = pd.DataFrame({'row': coo.row, 'col': coo.col, 'data': coo.data}
                 )[['row', 'col', 'data']].sort_values(['row', 'col']
                 ).reset_index(drop=True).groupby('col')
    df = pd.DataFrame(0 * np.ones((max(coo.row)+1, len(cols))), columns=cols, dtype=np.int64)
    tmp = coo_group.apply(lambda x:x.row.values)
    for col,rows,values in zip(tmp.index.tolist(),tmp.values,
                                coo_group.apply(lambda x:x.data.tolist())):
        df.iloc[rows,col]=values
    return df

In [30]:
gbm1 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0.9,
    subsample=0.8,
    colsample_bytree=0.75,
    objective='binary:logistic',
    nthread=-1,
    scale_pos_weight=4,
    seed=0)

In [42]:
gbm2 = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=500, 
    max_depth=3,
    min_child_weight=1,
    gamma=0.9,
    subsample=0.8,
    colsample_bytree=0.75,
    objective='binary:logistic',
    nthread=-1,
    scale_pos_weight=4,
    seed=0)

In [32]:
import os  
  
finish_mall = os.listdir('./model_negShop_dis/')
finish_mall.remove('.ipynb_checkpoints')
for i in range(len(finish_mall)):
    finish_mall[i] = finish_mall[i][:-2]
finish_mall

['m_623', 'm_6167', 'm_2878', 'm_4011', 'm_1831', 'm_4548', 'm_4187', 'm_4572']

In [33]:
len(finish_mall)

8

In [43]:
for mall_id in tqdm(mall_ids):
    if mall_id in finish_mall:
        train_new = []
        m_train = train[train.mall_id == mall_id]
        m_test = test[test.mall_id == mall_id]
        m_full = pd.concat([m_train, m_test])
        del m_train
        del m_test

        # shop_id onehot
#         shop = pd.get_dummies(m_full.shop_id)
        m_full = m_full.drop(['shop_id'], axis=1)
#         m_full = pd.concat([m_full, shop], axis=1)
#         del shop
        m_train = m_full[m_full.flag != -1]
        m_test = m_full[m_full.flag == -1]
        m_train = m_train.as_matrix()
        m_test = m_test.as_matrix()
        del m_full

        # wifi onehot
        m1 = mall_wifi_dateframe(mall_id, 'train_matrix')
        m2 = mall_wifi_dateframe(mall_id, 'test_matrix')
        m1 = m1.drop(['full_order'], axis=1)
        m2 = m2.drop(['full_order'], axis=1)
        wifi_spc = list(set(many_wifi).intersection(set(m1.columns)))
        m1 = m1[wifi_spc]
        m2 = m2[wifi_spc]
        m1 = m1.as_matrix()
        m2 = m2.as_matrix()
        m_train_new = []
        m_test_new = []
        for i in range(len(m1)):
            m_train_new.append(m1[i].tolist())
            m_train_new.append(m1[i].tolist())
            m_train_new.append(m1[i].tolist())
            m_train_new.append(m1[i].tolist())
            m_train_new.append(m1[i].tolist())
        for i in range(len(m2)):
            m_test_new.append(m2[i].tolist())
            m_test_new.append(m2[i].tolist())
            m_test_new.append(m2[i].tolist())
            m_test_new.append(m2[i].tolist())
            m_test_new.append(m2[i].tolist())
            m_test_new.append(m2[i].tolist())
            m_test_new.append(m2[i].tolist())
            m_test_new.append(m2[i].tolist())
            m_test_new.append(m2[i].tolist())
        del m1
        del m2
        m_trian_nn = np.hstack((m_train.tolist(), m_train_new))
        m_test_nn = np.hstack((m_test.tolist(), m_test_new))
        del m_train
        del m_test
        del m_train_new
        del m_test_new
        x_train = np.hstack((m_trian_nn[:, 2:4], m_trian_nn[:, 5:7], m_trian_nn[:, 8:]))
        y_train = m_trian_nn[:, 4]
        x_test = np.hstack((m_test_nn[:, 2:4], m_test_nn[:, 5:7], m_test_nn[:, 8:]))
        del m_trian_nn
        del m_test_nn
        
        x_train_tr = x_train[:-5000]
        y_train_tr = y_train[:-5000]
        x_train_val = x_train[-5000:]
        y_train_val = y_train[-5000:]
        
        x_train_1 = x_train_tr[-10000:]
        y_train_1 = y_train_tr[-10000:]
        x_train_2 = x_train_tr[:-10000]
        y_train_2 = y_train_tr[:-10000]

        gbm2.fit(x_train_1,  y_train_1)
        del x_train_1
        del y_train_1
        
        new_feature = gbm2.apply(x_train_2)
        new_feature_test = gbm2.apply(x_train_val)
        x_train_new2 = np.hstack((x_train_2, new_feature))
        x_test_new = np.hstack((x_train_val, new_feature_test))
        del new_feature
        del new_feature_test
        
        gbm1.fit(x_train_new2, y_train_2)
        
        y_pred_proba = gbm1.predict_proba(x_test_new)
        y_pred_proba = pd.DataFrame(y_pred_proba)
        y_max = []
        for i in range(y_pred_proba.shape[0]):
            if i % 5 == 0:
                temp = y_pred_proba.loc[i : i + 4, 1].sort_values(ascending=False)
                y_max.append(temp.index[0])
            else:
                y_max.append(0)
        y_max = pd.Series(y_max)
        acc = 1 - len(y_max[y_max % 5 != 0])/(len(y_train_val)/5)

        print(str(mall_id) + ':' + str(acc))
        finish_mall.append(mall_id)

  0%|          | 0/97 [00:00<?, ?it/s]

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.75,
       gamma=0.9, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=4, seed=0, silent=True, subsample=0.8)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.75,
       gamma=0.9, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=4, seed=0, silent=True, subsample=0.8)

  3%|▎         | 3/97 [00:51<26:42, 17.05s/it]

m_623:0.794


...

m_4548:0.864


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.75,
       gamma=0.9, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=4, seed=0, silent=True, subsample=0.8)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.75,
       gamma=0.9, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=4, seed=0, silent=True, subsample=0.8)

100%|██████████| 97/97 [09:28<00:00,  5.86s/it]

m_4011:0.864


In [ ]:
# train_neg changeShop

In [ ]:
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.75,
       gamma=0.9, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=4, seed=0, silent=1, subsample=0.8)
m_623:0.81
m_4187:0.661
m_1831:0.8160000000000001
m_4572:0.734
m_6167:0.621
m_2878:0.75
m_4548:0.873
m_4011:0.867